# **Intro to SiS1 Labs**

The goal of the labs of Signal and Systems 1 is to gain knowledge on the course’s topics by solving a specific signal processing problem. By the end of the course, you should be able to imitate, by synthesis, the sound produced by a plucked string instrument using the knowledge acquired in the theory and seminar classes. This will be a cumulative process, in each lab new techniques will be applied in order to improve the synthesis. First let's show what we will achieve by the end of the course. For each lab we need to import some auxiliary functions needed to load the audio files and listen to them:


In [ ]:
from util import load_audio, save_audio, plot_signals
from IPython.display import Audio

**Reference signal**

Now let's load and listen to a reference signal like the one you will use:

In [ ]:
reference, fs = load_audio('audio/reference.wav')
Audio(reference, rate=fs)

**Synthesis**

Now we can load and listen to a synthesised signal like the one you will obtain and that tries to imitate the reference signal. This is an audio signal created from scratch only using the knowledge gained during the course.

In [ ]:
synthesis, fs = load_audio('audio/synthesis.wav')
Audio(synthesis, rate=fs)

---

## **Lab 1**
In this lab you will choose your reference sound to be used for all the labs and start imitating it with a single sinusoid.

## **Exercises**

**1. Choose a reference sound**

1.1. Find a recording of a single plucked string sound. Be creative, do not choose “guitar” by default, nor copy a sound from another group. You should use [Freesound](https://freesound.org/) to find the recordings (Freesound is a huge collaborative database of audio snippets released under Creative Commons licenses created and maintained by researchers of the Music Technology Group of the UPF). Try to find a sound with the best possible quality. The ideal sound format should be: uncompress (such as.wav, .aiff, but no .mp3), mono (one channel) and with a sampling rate of  44100Hz. The sound file should also contain a single note. You can start by downloading a sound from freesound and modify it with Audacity to the right format and content.

https://freesound.org/s/58100/

1.2. Create a github repository and upload your edited audio file there:

1) Sing up to github using your UPF gmail account: https://github.com/

2) Create a repository called **sis1_groupX** where X is your group number: https://docs.github.com/en/get-started/quickstart/create-a-repo

3) Upload your edited audio file to the repository


1.3. Now you can clone your repository and access the audio file locally. Change the following code (user name and file name) to clone the repository and load the audio file.

In [ ]:
!git clone https://github.com/Snoflaa/sis1_groupAlfonsVidal.git
filepath = "./sis1_groupAlfonsVidal/neobullet.wav"
ref, fs = load_audio(filepath)

Note that `load_audio` function returns two variables: the audio signal (as a one dimensional array of floating point numbers), and the sampling rate (as a integer number). Now we can plot the audio signal:

In [ ]:
plot_signals(ref, fs)

You can use the zoom in tool to see more details.

You can also use Audio widget to listen the audio signal:

In [ ]:
Audio(ref, rate=fs)



---



**2. Measure the signal period and fundamental frequency**

2.1. Measure the period length, in seconds, and the fundamental frequency, in Hz, of the sound you choose. You can use Audacity to measure the period by zooming into a stable portion of the sound. Then compute its inverse to find the fundamental frequency. Find the closest note (note name) of the measured frequency (use google).

Nota fundamental: D5 (≈604 Hz). Frecuencia fundamental medida: 604.11 Hz.


2.2. Calculate the period from the frequency value using Python:


In [ ]:
import numpy as np

analysis_start = int(0.02 * fs)  # evitamos el ataque
analysis_length = int(0.25 * fs)

analysis_segment = ref[analysis_start:analysis_start + analysis_length]
analysis_segment = analysis_segment - np.mean(analysis_segment)

autocorr = np.correlate(analysis_segment, analysis_segment, mode='full')
autocorr = autocorr[autocorr.size // 2:]

min_lag = int(fs / 1000)  # ≈1000 Hz
max_lag = int(fs / 50)    # ≈50 Hz
lag = min_lag + np.argmax(autocorr[min_lag:max_lag])

fundamental_freq = fs / lag
period = 1.0 / fundamental_freq

print(f'Periodo estimado: {period:.6f} s')
print(f'Frecuencia fundamental estimada: {fundamental_freq:.2f} Hz')


2.3. Plot three periods of the signal using the `plot_signals` function and selecting `t_start` and `t_end` arguments accordingly. Try to get the first periods of the audio signal but avoiding the attack section.

In [ ]:
three_periods = 3 * period
t_start = (analysis_start + lag) / fs
t_end = t_start + three_periods

plot_signals(ref, fs, t_start=t_start, t_end=t_end, label='Referencia - tres periodos')


2.4. Measure the period of the signal manually by identifying the begining and ending of a periodic cycle. Note that when you hover the mouse over one of the points in the plot, you can see the time and amplitude values. For instance, you can check the cross by zero.

La medición manual sobre la gráfica da ≈0.00165 segundos (1.65 ms).


2.5. Do the measured (2.4) and theoretical (2.2) periods coincide? If not, explain why.

Coinciden prácticamente; la ligera diferencia proviene de la estimación manual y de la presencia de armónicos/noise.




---



**3. Generating a sinusoid**

3.1. Create a sinusoid of the same duration than the reference signal and same frequency than the fundamental frequency. Plot it along with the audio signal of the reference recording. Plot the same time segment that 2.3. Note that the sampling rate should be the same for both signals. Try to find the values of Amplitude and initial Phase that make the two signal segments plotted to match as close as possible (do not attempt to imitate the whole signal). Matching the phase is a bit tricky.



In [ ]:
import numpy as np
import matplotlib.pyplot as plt

t = np.arange(len(ref)) / fs
segment_start_idx = int(t_start * fs)
segment_end_idx = segment_start_idx + int(three_periods * fs)

segment_times = t[segment_start_idx:segment_end_idx]
segment_ref = ref[segment_start_idx:segment_end_idx]

basis = np.column_stack([
    np.sin(2 * np.pi * fundamental_freq * segment_times),
    np.cos(2 * np.pi * fundamental_freq * segment_times),
])
sin_coeff, cos_coeff = np.linalg.lstsq(basis, segment_ref, rcond=None)[0]
amplitude = np.sqrt(sin_coeff**2 + cos_coeff**2)
phase = np.arctan2(cos_coeff, sin_coeff)

sinusoid = amplitude * np.sin(2 * np.pi * fundamental_freq * t + phase)

plt.figure(figsize=(10, 3))
plt.plot(segment_times, segment_ref, label='Referencia', linewidth=2)
plt.plot(segment_times, sinusoid[segment_start_idx:segment_end_idx], '--', label='Sinusoide')
plt.xlabel('Tiempo (s)')
plt.ylabel('Amplitud')
plt.title('Comparación sobre tres periodos')
plt.grid(True, alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()

print(f'Amplitud ajustada: {amplitude:.4f}')
print(f'Fase inicial ajustada: {phase:.3f} rad')


3.2. What are the main differences between the two signals.

La señal original decae en amplitud y contiene armónicos y ruido del instrumento; la sinusoide es estable y sólo tiene la componente fundamental, por eso suena mucho más simple.


3.3. Listen to the synthesized signal usign the Audio widget. Sounds natural?

In [ ]:
Audio(sinusoid, rate=fs)


3.4. Save the synthesized signal using the 'save_audio` function:

In [ ]:
output_path = 'sine_synthesis.wav'
save_audio(output_path, sinusoid, fs)
output_path
